In [1]:
#coding:utf8
import os
from PIL import  Image
from torch.utils import data
import numpy as np
from torchvision import transforms as T
import xml.etree.ElementTree as ET
import torch as t
from nltk.tokenize import word_tokenize

对于Attention-based LSTM<br>
每组数据需要<br>
word representation $w_{1}, w_{2}, ..., w_{N}$<br>
aspect embedding $v_{\alpha}$<br>
polarity $p$

In [1]:
class AspClas(data.Dataset):
    
    def __init__(self, root, embedrt, transforms=None, train=True, test=False):
        '''
        主要目标：
        载入文件 解析数据
        根据 训练、验证、测试 划分数据
        '''
        self.raw_cases = []
        self.cases = []
        
        # root = 'restaurants-trial.xml'
        xml = ET.parse(root)
        for s in xml.findall('sentence'):
            if s.find('aspectTerms'):
                text = s.find('text').text
                asps = s.find('aspectTerms').findall('aspectTerm')
                for asp in asps:
                    self.raw_cases.append((text, asp.attrib['term'], asp.attrib['polarity']))
        
        # shuffle
        np.random.seed(100)
        self.raw_cases = np.random.permutation(self.raw_cases)
        
        # division
        if test:
            pass
        elif train:
            self.raw_cases = self.raw_cases[:int(0.7*len(self.raw_cases))]
        else:
            self.raw_cases = self.raw_cases[int(0.7*len(self.raw_cases)):]
        
        # transform
        self.embedding(embedrt)
        
        return
    
    def __getitem__(self, index):
        '''
        一次返回一个 sentence-term-polarity
        '''
        case = self.cases[index]
        return case[0], case[1], case[2]
    
    def __len__(self):
        return len(self.cases)
    
    def embedding(self, root):
        '''
        transform the strings into vectors using pre-trained word vectors
        '''
        
        # load file and init dict
        self.embed = {}
        self.embed['not found'] = t.Tensor(np.zeros(300), dtype=float)
        
        # root = 'glove.840B.300d/glove.840B.300d.txt'
        f = open(root, 'r')
        l = f.readline()
        while(l != ''):
            if l [-1] == '\n':
                l = l[:-1]
            l = l.split(' ')
            self.embed[l[0]] = t.Tensor(np.array(l[1:], dtype=float))
            l = f.readline()
        
        embedks = self.embed.keys()
        self.cases = []
        self.polar = {'positive':1, 'neutral':0, 'negative':-1}
        
        # convert the strings in self.raw_cases into tensors
        for (raw_text, raw_term, raw_polarity) in self.raw_cases:
            # text
            raw_words = word_tokenize(raw_text)
            text = []
            for rw in raw_words:
                w = self.embed[rw] if rw in embedks else self.embed['not found']
                text.append(w)
            # term
            term = self.embed[raw_term] if raw_term in embedks else self.embed['not found']
            # polarity
            polarity = self.polar[raw_polarity]
            
            self.cases.append((text, term, polarity))
        self.raw_cases = []
        
        return

NameError: name 'data' is not defined

In [6]:
a = object()

In [9]:
import sys
sys.path.append('D:\\Jupyter\\Python\\ATAE-LSTM')
import Ipynb_importer
from config import opt
from Embedding import emb